In [15]:
import sqlite3 as sql
import pandas as pd

In [16]:
# create and make a connection to the database file
conn = sql.connect('dojo.db')

In [17]:
# we will use pandas for easier file loading to the database
csv_files = ['location', 'region', 'product', 'location_region', 'sales']
for csv in csv_files:
    df = pd.read_csv(f'.//db_files//{csv}.csv')
    df.to_sql(csv, conn, if_exists='replace', index=False)

In [18]:
# cursor is what is used to "drive" queries
c = conn.cursor()

In [19]:
# now that we have the database set up, we can query our data

# simple query to pull all locations and their regions
c.execute('select * from location_region')
# fetchall retreives the results of the most recent execute statement
print(c.fetchall())

[('L001', 'R010'), ('L002', 'R005'), ('L003', 'R009'), ('L004', 'R001'), ('L005', 'R005'), ('L006', 'R006'), ('L007', 'R003'), ('L008', 'R003'), ('L009', 'R010'), ('L010', 'R010'), ('L011', 'R001'), ('L012', 'R005'), ('L013', 'R002'), ('L014', 'R002'), ('L015', 'R004'), ('L016', 'R004'), ('L017', 'R003'), ('L018', 'R009'), ('L019', 'R006'), ('L020', 'R003'), ('L021', 'R006'), ('L022', 'R002'), ('L023', 'R002'), ('L024', 'R010'), ('L025', 'R004'), ('L026', 'R005'), ('L027', 'R004'), ('L028', 'R001'), ('L029', 'R006'), ('L030', 'R007'), ('L031', 'R005'), ('L032', 'R007'), ('L033', 'R003'), ('L034', 'R004'), ('L035', 'R002'), ('L036', 'R009'), ('L037', 'R008'), ('L038', 'R007'), ('L039', 'R010'), ('L040', 'R006'), ('L041', 'R010'), ('L042', 'R004'), ('L043', 'R003'), ('L044', 'R009'), ('L045', 'R005'), ('L046', 'R006'), ('L047', 'R003'), ('L048', 'R008'), ('L049', 'R003'), ('L050', 'R002'), ('L051', 'R005')]


In [27]:
# let's pull the names for the locations and regions instead of the ids
# for this we need to join the location, region, and location_region table 
# and use the where clause to show the feild that are the same 
c.execute("""
    select location_name, region_name
    from location join region join location_region
    where location.location_id == location_region.location_id
        and region.region_id == location_region.region_id    
""")
print(c.fetchall())

[('California', 'Far West'), ('Illinois', 'Great Lakes'), ('Delaware', 'Midsouth'), ('Iowa', 'Midwest'), ('Arizona', 'Mountain West'), ('Connecticut', 'New England'), ('New Jersey', 'Northeast'), ('Oregon', 'Northwest'), ('Arkansas', 'South Central'), ('Alabama', 'Southeast')]


In [21]:
# now we will look at some sales data
# similar to the last query we will join the relevant tables and 
# show which field are the same
c.execute("""
    select location_name, product_name, lbs, price_per_lb
    from location join location_region join sales join product
    where location.location_id == location_region.location_id
        and location.location_id == sales.location_id
        and sales.product_id = product.product_id 
""")
print(c.fetchmany(20)) # just pull the first 20 (not the top 20; we will do that next)

[('Alabama', 'Alfalfa pellets', 707.2, 4.68), ('Alabama', 'Apples, dried', 639.3, 4.63), ('Alabama', 'Chestnuts', 828.9, 1.07), ('Alabama', 'Chilled meat', 158.0, 1.21), ('Alabama', 'Corn', 712.1, 4.38), ('Alabama', 'Fabrics/cloth/textiles', 2.2, 3.05), ('Alabama', 'Flax tow', 916.7, 3.13), ('Alabama', 'Household appliances/white goods', 875.4, 1.84), ('Alabama', 'Kiwifruit', 843.3, 2.56), ('Alabama', 'Onions', 790.0, 4.46), ('Alabama', 'Palm kernel oil', 744.9, 2.89), ('Alabama', 'Peanut expeller', 795.4, 5.43), ('Alabama', 'Potatoes', 334.1, 1.76), ('Alabama', 'Raw sugar', 204.0, 3.26), ('Alabama', 'Roofing felt', 436.3, 2.61), ('Alabama', 'Sesame seed expeller', 88.4, 3.17), ('Alabama', 'Sunflower pellets', 83.4, 4.58), ('Alabama', 'Sunflower pellets', 972.8, 4.58), ('Alabama', 'Wool', 442.3, 4.5), ('Arizona', 'Avocados', 159.6, 2.62)]


In [24]:
# let's now look at the total price and sort the values
c.execute("""
    select location_name, product_name, lbs, price_per_lb, (lbs * price_per_lb) price
    from location join location_region join sales join product
    where location.location_id == location_region.location_id
        and location.location_id == sales.location_id
        and sales.product_id = product.product_id 
    order by price desc
""")
print(c.fetchmany(20)) # just pull the first 20 (not the top 20; we will do that next)

[('Ohio', 'Almonds', 835.7, 5.89, 4922.273), ('Oklahoma', 'Apples', 961.2, 5.11, 4911.732000000001), ('North Dakota', 'Almonds', 800.4, 5.89, 4714.356), ('Kansas', 'Carpets', 764.5, 5.76, 4403.5199999999995), ('Pennsylvania', 'Apples, dried', 893.4, 4.63, 4136.442), ('Kentucky', 'Alfalfa pellets', 746.5, 4.68, 3493.62), ('Texas', 'Almonds', 591.2, 5.89, 3482.168), ('South Carolina', 'Almonds', 585.9, 5.89, 3450.9509999999996), ('Alabama', 'Alfalfa pellets', 707.2, 4.68, 3309.696), ('Illinois', 'Barley', 714.5, 4.58, 3272.41), ('North Carolina', 'Beans, dried', 632.0, 4.96, 3134.72), ('Arkansas', 'Alfalfa pellets', 624.6, 4.68, 2923.1279999999997), ('Colorado', 'Almonds', 484.1, 5.89, 2851.349), ('Rhode Island', 'Bay leaves', 640.0, 4.28, 2739.2000000000003), ('New Mexico', 'Limes', 445.0, 5.94, 2643.3), ('Georgia', 'Capsicum', 553.7, 4.61, 2552.5570000000002), ('New York', 'Asparagus', 666.6, 3.74, 2493.0840000000003), ('Louisiana', 'Brewer�s grain pellets', 925.0, 2.66, 2460.5), ('Con

In [9]:
# we can print this a little prettier
print(f'Name, Product, Price')
lines = c.fetchmany(50)
for line in lines:
    print(f'{line[0]}, {line[1]}, ${line[4]:.2f}')

Name, Product, Price
Kentucky, Candy sugar, $4776.49
Wyoming, Almonds, $4749.70
North Dakota, Almonds, $4714.36
North Carolina, Mangos, $4711.27
Indiana, Peanut expeller, $4666.54
Florida, Chocolate, solid, $4635.90
Washington, Artichokes, $4630.75
Kansas, Oranges, $4558.18
New Hampshire, Wheat bran pellets, $4504.56
Florida, Sesame oil, $4474.25
Alabama, Sunflower pellets, $4455.42
Iowa, Ramie, $4454.16
Kansas, Carpets, $4403.52
Colorado, Copra expeller, $4402.48
Wisconsin, Caraway, $4396.30
California, Mustard oil, $4364.98
Alabama, Peanut expeller, $4319.02
Arizona, Chocolate, solid, $4286.04
West Virginia, Hemp oil, $4282.20
South Carolina, Palm oil, $4253.59
Virginia, Frozen meat, $4209.03
Kentucky, Bay leaves, $4195.68
Ohio, Palm oil, $4172.94
Washington, Newsprint, $4168.63
District of Columbia, Sunflower pellets, $4136.66
Pennsylvania, Apples, dried, $4136.44
Kansas, Sunflower pellets, $4036.35
New Hampshire, Cottonseed oil, $4017.66
Puerto Rico, Peanut expeller, $3995.94
Iowa,

In [14]:
# last query we will take some user input for a region and how the sales by product
# lets start by getting the available regions
c.execute('select region_name from region')
regions = [reg[0] for reg in c.fetchall()] # fetchall returns tuple of lists, convert to just a list
region = ''
while True:
    print('Avaiable regions: ' + ', '.join(regions)) # show the valid regions
    region = input('What input would you like to see sales data for? ') # ask user for input
    # if the input is valid, we can break the loop, else ask for input again
    if region not in regions:
        print("Invalid Input!")
    else:
        break

# query the database using the user's input
# we need to introduce the group by clause since the products are 
# associated with with locations, not regions
# we also will use the where clause to test for only the region we want
c.execute("""
    select region_name, product_name, (lbs * price_per_lb) price
    from region join location_region join sales join product
    where region.region_id == location_region.region_id
        and location_region.location_id == sales.location_id
        and sales.product_id = product.product_id 
        and region_name = ?
    order by price desc
""", (region,)) # when only using one parameter in the query you need an extra comma and the end of the tuple being passed in

results = c.fetchall()
print(f'Results for {region}\nProduct, Price')
print('-'*30)
for line in results:
    print(f'{line[1]}, ${line[2]:.2f}')

Avaiable regions: Far West, Great Lakes, Midsouth, Midwest, Mountain West, New England, Northeast, Northwest, South Central, Southeast
Results for Far West
Product, Price
------------------------------
Natural rubber, $4975.56
Mustard oil, $4364.98
Limes, $3776.06
Linseed oil, $3681.40
Wool, $3438.00
Sunflower pellets, $3244.01
Rice bran pellets, $2578.33
Tomatoes, $2508.32
Copra, $2504.90
Preserved foods, general, $2400.55
Sage leaves, $2305.51
Raffia, $2297.46
Cucumbers, $2260.90
Absorbent cotton, $2233.11
Pepper, $2151.19
Wire rod, $2025.38
Sesame oil, $1965.15
Natural rubber, $1799.44
Peanut oil, $1731.59
Cherries, $1627.92
Photographic paper, $1456.32
Sunflower oil, $1411.93
Mustard oil, $1273.27
Peanut oil, $1015.10
Kapok seed oil, $986.46
Mandarins, $933.41
Cinnamon, $909.44
Garlic, $846.50
Coconut oil, $828.52
Apricots, dried, $676.09
Castor oil, $622.75
Coriander, $537.89
Chilled meat, $371.83
Synthetic rubber, $336.44
Cashew nuts, $176.67
Linseed oil, $145.39
Peaches/nectarin

In [ ]:
# last step we need is to close the connection to the database
conn.close()